# Machine Learning Library (MLlib)
MLlib is Spark’s machine learning (ML) library. Its goal is to make practical machine learning scalable and easy. At a high level, it provides tools such as:

- ML Algorithms: common learning algorithms such as classification, regression, clustering, and collaborative filtering
- Featurization: feature extraction, transformation, dimensionality reduction, and selection
- Pipelines: tools for constructing, evaluating, and tuning ML Pipelines
- Persistence: saving and load algorithms, models, and Pipelines
- Utilities: linear algebra, statistics, data handling, etc.

As of Spark 2.0, the RDD-based APIs in the spark.mllib package have entered maintenance mode. The primary Machine Learning API for Spark is now the DataFrame-based API in the spark.ml package.

# An example classification model
The objective is to learn how to build a complete classification workflow from the beginning to the end.
Problem Definition.  The problem we are going to solve is the infamous Titanic Survival Problem. We are asked to build a machine learning model that takes passenger information and predict whether he/she survived or not. The dataset contains 12 columns described as follows: 
![Titanic data description](titanic.png)


# Step 1: Load Data

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
data = spark.read.csv('titanic.csv', inferSchema=True, header=True)
data.show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|Gender| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|
|          6|       0|     3|    Moran, Mr. James|  male|null|    0|    0|      

Let us filter out the unneeded columns:

In [2]:
data = data.select(['Survived', 'Pclass', 'Gender', 'Age', 'SibSp', 'Parch', 'Fare'])
data.printSchema()
data.head(3) # this is an action. Head() returns a list of rows.

root
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Fare: double (nullable = true)



[Row(Survived=0, Pclass=3, Gender='male', Age=22.0, SibSp=1, Parch=0, Fare=7.25),
 Row(Survived=1, Pclass=1, Gender='female', Age=38.0, SibSp=1, Parch=0, Fare=71.2833),
 Row(Survived=1, Pclass=3, Gender='female', Age=26.0, SibSp=0, Parch=0, Fare=7.925)]

# Step 2. Data Exploration

In [3]:
data.describe().show() # show basic stats that is very useful for continuous variables.

+-------+-------------------+------------------+------+------------------+------------------+-------------------+-----------------+
|summary|           Survived|            Pclass|Gender|               Age|             SibSp|              Parch|             Fare|
+-------+-------------------+------------------+------+------------------+------------------+-------------------+-----------------+
|  count|                891|               891|   891|               714|               891|                891|              891|
|   mean| 0.3838383838383838| 2.308641975308642|  null| 29.69911764705882|0.5230078563411896|0.38159371492704824| 32.2042079685746|
| stddev|0.48659245426485753|0.8360712409770491|  null|14.526497332334035|1.1027434322934315| 0.8060572211299488|49.69342859718089|
|    min|                  0|                 1|female|              0.42|                 0|                  0|              0.0|
|    max|                  1|                 3|  male|              80.0|  

In [4]:
# code to show value count of each variable. This is very useful for categorical variables. 
for column in data.columns:
    data.groupBy(column).count().show()

+--------+-----+
|Survived|count|
+--------+-----+
|       1|  342|
|       0|  549|
+--------+-----+

+------+-----+
|Pclass|count|
+------+-----+
|     1|  216|
|     3|  491|
|     2|  184|
+------+-----+

+------+-----+
|Gender|count|
+------+-----+
|female|  314|
|  male|  577|
+------+-----+

+----+-----+
| Age|count|
+----+-----+
| 8.0|    4|
|70.0|    2|
| 7.0|    3|
|20.5|    1|
|49.0|    6|
|29.0|   20|
|40.5|    2|
|64.0|    2|
|47.0|    9|
|42.0|   13|
|24.5|    1|
|44.0|    9|
|35.0|   18|
|null|  177|
|62.0|    4|
|18.0|   26|
|80.0|    1|
|34.5|    1|
|39.0|   14|
| 1.0|    7|
+----+-----+
only showing top 20 rows

+-----+-----+
|SibSp|count|
+-----+-----+
|    1|  209|
|    3|   16|
|    5|    5|
|    4|   18|
|    8|    7|
|    2|   28|
|    0|  608|
+-----+-----+

+-----+-----+
|Parch|count|
+-----+-----+
|    1|  118|
|    6|    1|
|    3|    5|
|    5|    5|
|    4|    4|
|    2|   80|
|    0|  678|
+-----+-----+

+-------+-----+
|   Fare|count|
+-------+-----+
| 8.

# Step 3. Feature Transformation

## Missing value imputation

### If a continuous variable contains missing values, we do missing value imputation in two steps: 1) We add a missing value indicator; and 2) we replace missing values with the mean

In [5]:
from pyspark.sql import functions as f
data = data.withColumn('AgeMissing', f.when(f.isnull(data['age']), 1).otherwise(0))
data.groupBy("AgeMissing").count().show()
data.show()
from pyspark.ml.feature import Imputer
imputer = Imputer(strategy='mean', inputCols=['Age'], outputCols=['AgeImputed'])
imputer_model = imputer.fit(data)
data = imputer_model.transform(data)
data.show()

+----------+-----+
|AgeMissing|count|
+----------+-----+
|         1|  177|
|         0|  714|
+----------+-----+

+--------+------+------+----+-----+-----+-------+----------+
|Survived|Pclass|Gender| Age|SibSp|Parch|   Fare|AgeMissing|
+--------+------+------+----+-----+-----+-------+----------+
|       0|     3|  male|22.0|    1|    0|   7.25|         0|
|       1|     1|female|38.0|    1|    0|71.2833|         0|
|       1|     3|female|26.0|    0|    0|  7.925|         0|
|       1|     1|female|35.0|    1|    0|   53.1|         0|
|       0|     3|  male|35.0|    0|    0|   8.05|         0|
|       0|     3|  male|null|    0|    0| 8.4583|         1|
|       0|     1|  male|54.0|    0|    0|51.8625|         0|
|       0|     3|  male| 2.0|    3|    1| 21.075|         0|
|       1|     3|female|27.0|    0|    2|11.1333|         0|
|       1|     2|female|14.0|    1|    0|30.0708|         0|
|       1|     3|female| 4.0|    1|    1|   16.7|         0|
|       1|     1|female|58.0|  

## Categorical variable encoding:
We learned that machine learning algorithms cannot deal with categorical features. So, we need to index the Gender values:

In [6]:
from pyspark.ml.feature import StringIndexer
gender_indexer = StringIndexer(inputCol='Gender', outputCol='GenderIndexed')
gender_indexer_model = gender_indexer.fit(data)
data = gender_indexer_model.transform(data)
data.show()

+--------+------+------+----+-----+-----+-------+----------+-----------------+-------------+
|Survived|Pclass|Gender| Age|SibSp|Parch|   Fare|AgeMissing|       AgeImputed|GenderIndexed|
+--------+------+------+----+-----+-----+-------+----------+-----------------+-------------+
|       0|     3|  male|22.0|    1|    0|   7.25|         0|             22.0|          0.0|
|       1|     1|female|38.0|    1|    0|71.2833|         0|             38.0|          1.0|
|       1|     3|female|26.0|    0|    0|  7.925|         0|             26.0|          1.0|
|       1|     1|female|35.0|    1|    0|   53.1|         0|             35.0|          1.0|
|       0|     3|  male|35.0|    0|    0|   8.05|         0|             35.0|          0.0|
|       0|     3|  male|null|    0|    0| 8.4583|         1|29.69911764705882|          0.0|
|       0|     1|  male|54.0|    0|    0|51.8625|         0|             54.0|          0.0|
|       0|     3|  male| 2.0|    3|    1| 21.075|         0|          

### You still need to deal with ordinal/nominal variables - You need to do dummy coding!

In [7]:
from pyspark.ml.feature import OneHotEncoderEstimator
encoder = OneHotEncoderEstimator(inputCols=["Pclass"],
                                 outputCols=["PclassVec"])
model = encoder.fit(data)
data = model.transform(data)
data.show()

+--------+------+------+----+-----+-----+-------+----------+-----------------+-------------+-------------+
|Survived|Pclass|Gender| Age|SibSp|Parch|   Fare|AgeMissing|       AgeImputed|GenderIndexed|    PclassVec|
+--------+------+------+----+-----+-----+-------+----------+-----------------+-------------+-------------+
|       0|     3|  male|22.0|    1|    0|   7.25|         0|             22.0|          0.0|    (3,[],[])|
|       1|     1|female|38.0|    1|    0|71.2833|         0|             38.0|          1.0|(3,[1],[1.0])|
|       1|     3|female|26.0|    0|    0|  7.925|         0|             26.0|          1.0|    (3,[],[])|
|       1|     1|female|35.0|    1|    0|   53.1|         0|             35.0|          1.0|(3,[1],[1.0])|
|       0|     3|  male|35.0|    0|    0|   8.05|         0|             35.0|          0.0|    (3,[],[])|
|       0|     3|  male|null|    0|    0| 8.4583|         1|29.69911764705882|          0.0|    (3,[],[])|
|       0|     1|  male|54.0|    0|  

### Creating the Features Vector
MLlib expects data to be represented in two columns: a features vector and a label column. We have the label column ready (Survived), so let us prepare the features vector.

In [8]:
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=['SibSp', 'Parch', 'Fare', 'AgeMissing', 'AgeImputed', 'GenderIndexed', 'PclassVec'], outputCol='features')
data = assembler.transform(data)
data.show()
data.head(5)

+--------+------+------+----+-----+-----+-------+----------+-----------------+-------------+-------------+--------------------+
|Survived|Pclass|Gender| Age|SibSp|Parch|   Fare|AgeMissing|       AgeImputed|GenderIndexed|    PclassVec|            features|
+--------+------+------+----+-----+-----+-------+----------+-----------------+-------------+-------------+--------------------+
|       0|     3|  male|22.0|    1|    0|   7.25|         0|             22.0|          0.0|    (3,[],[])|(9,[0,2,4],[1.0,7...|
|       1|     1|female|38.0|    1|    0|71.2833|         0|             38.0|          1.0|(3,[1],[1.0])|[1.0,0.0,71.2833,...|
|       1|     3|female|26.0|    0|    0|  7.925|         0|             26.0|          1.0|    (3,[],[])|(9,[2,4,5],[7.925...|
|       1|     1|female|35.0|    1|    0|   53.1|         0|             35.0|          1.0|(3,[1],[1.0])|[1.0,0.0,53.1,0.0...|
|       0|     3|  male|35.0|    0|    0|   8.05|         0|             35.0|          0.0|    (3,[],[]

[Row(Survived=0, Pclass=3, Gender='male', Age=22.0, SibSp=1, Parch=0, Fare=7.25, AgeMissing=0, AgeImputed=22.0, GenderIndexed=0.0, PclassVec=SparseVector(3, {}), features=SparseVector(9, {0: 1.0, 2: 7.25, 4: 22.0})),
 Row(Survived=1, Pclass=1, Gender='female', Age=38.0, SibSp=1, Parch=0, Fare=71.2833, AgeMissing=0, AgeImputed=38.0, GenderIndexed=1.0, PclassVec=SparseVector(3, {1: 1.0}), features=DenseVector([1.0, 0.0, 71.2833, 0.0, 38.0, 1.0, 0.0, 1.0, 0.0])),
 Row(Survived=1, Pclass=3, Gender='female', Age=26.0, SibSp=0, Parch=0, Fare=7.925, AgeMissing=0, AgeImputed=26.0, GenderIndexed=1.0, PclassVec=SparseVector(3, {}), features=SparseVector(9, {2: 7.925, 4: 26.0, 5: 1.0})),
 Row(Survived=1, Pclass=1, Gender='female', Age=35.0, SibSp=1, Parch=0, Fare=53.1, AgeMissing=0, AgeImputed=35.0, GenderIndexed=1.0, PclassVec=SparseVector(3, {1: 1.0}), features=DenseVector([1.0, 0.0, 53.1, 0.0, 35.0, 1.0, 0.0, 1.0, 0.0])),
 Row(Survived=0, Pclass=3, Gender='male', Age=35.0, SibSp=0, Parch=0, Fa

# Step 4. Training vs. test dataset partitioning

In [9]:
train, test = data.randomSplit([0.8, 0.2], seed=12345)

# Step 5. Training

In [10]:
# here we use Randomforest
from pyspark.ml.classification import RandomForestClassifier
algo = RandomForestClassifier(featuresCol='features', labelCol='Survived')
model = algo.fit(train)

# Step 6. Model Evaluation

In [11]:
# make prediction using test data
predictions = model.transform(test)
predictions.select(['Survived','prediction', 'probability']).show()

+--------+----------+--------------------+
|Survived|prediction|         probability|
+--------+----------+--------------------+
|       0|       0.0|[0.84005078342312...|
|       0|       0.0|[0.65774691756379...|
|       0|       0.0|[0.65774691756379...|
|       0|       0.0|[0.70034897625191...|
|       0|       0.0|[0.70034897625191...|
|       0|       0.0|[0.69932266046244...|
|       0|       0.0|[0.61705243035456...|
|       0|       0.0|[0.58924865072409...|
|       0|       0.0|[0.79303504454620...|
|       0|       0.0|[0.61363329549109...|
|       0|       0.0|[0.67153759366521...|
|       0|       0.0|[0.65853261591172...|
|       0|       0.0|[0.65853261591172...|
|       0|       0.0|[0.65736176948938...|
|       0|       0.0|[0.70037620775897...|
|       0|       0.0|[0.68212453041439...|
|       0|       1.0|[0.26752852959931...|
|       0|       0.0|[0.88108177084285...|
|       0|       0.0|[0.82112991216259...|
|       0|       0.0|[0.71016503428457...|
+--------+-

### In MLlib the default metric used for evaluating classification models is area_under_roc (auc)

In [12]:
# using spark ml to do model evaluation 
from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluator = BinaryClassificationEvaluator(labelCol='Survived', metricName='areaUnderROC')
evaluator.evaluate(predictions)

0.8863015138772078

### Model Evaluation with SciKit-Learn
If you want to generate other evaluations such as a confusion matrix or a classification report, you could always use the scikit-learn 

In [13]:
y_true = predictions.select(['Survived']).collect()
y_pred = predictions.select(['prediction']).collect()
from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       0.82      0.91      0.87       116
           1       0.86      0.72      0.78        82

    accuracy                           0.83       198
   macro avg       0.84      0.82      0.82       198
weighted avg       0.84      0.83      0.83       198



In [14]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
data1 = spark.read.csv('titanic.csv', inferSchema=True, header=True)
data1 = data1.withColumn('AgeMissing', f.when(f.isnull(data1['age']), 1).otherwise(0))
age_imputer = Imputer(strategy='mean', inputCols=['Age'], outputCols=['AgeImputed'])
gender_indexer = StringIndexer(inputCol='Gender', outputCol='GenderIndexed')
pclass_encoder = OneHotEncoderEstimator(inputCols=["Pclass"],outputCols=["PclassVec"])
assembler = VectorAssembler(inputCols=['SibSp', 'Parch', 'Fare', 'AgeMissing', 'AgeImputed', 'GenderIndexed', 'PclassVec'], outputCol='features')
algo = RandomForestClassifier(featuresCol='features', labelCol='Survived')
# chain the steps in a piperline
from pyspark.ml import Pipeline
pipeline = Pipeline(stages=[age_imputer, gender_indexer, pclass_encoder, assembler, algo]) 
train1, test1 = data1.randomSplit([0.8, 0.2], seed=12345)
# Train model.  This also runs the indexers.
model = pipeline.fit(train1)
# Predictions
predictions = model.transform(test1)
predictions.select(['Survived','prediction', 'probability']).show()
evaluator = BinaryClassificationEvaluator(labelCol='Survived', metricName='areaUnderROC')
evaluator.evaluate(predictions)

+--------+----------+--------------------+
|Survived|prediction|         probability|
+--------+----------+--------------------+
|       0|       0.0|[0.87805007925591...|
|       0|       0.0|[0.52932263083207...|
|       1|       1.0|[0.41824239022645...|
|       1|       1.0|[0.24887939411156...|
|       0|       0.0|[0.67944652654420...|
|       1|       0.0|[0.87721640405769...|
|       0|       0.0|[0.54062209225734...|
|       0|       0.0|[0.59965089522779...|
|       0|       0.0|[0.87989037427045...|
|       0|       0.0|[0.87805007925591...|
|       1|       1.0|[0.17182219123661...|
|       1|       1.0|[0.17182219123661...|
|       0|       0.0|[0.69766666817993...|
|       1|       0.0|[0.70603093171371...|
|       0|       0.0|[0.87989037427045...|
|       1|       1.0|[0.44194310571070...|
|       0|       0.0|[0.87805007925591...|
|       1|       1.0|[0.26178465297380...|
|       0|       0.0|[0.77291472359105...|
|       0|       0.0|[0.87989037427045...|
+--------+-

0.8554656780137061